In [1]:
import os
import shutil
from pathlib import Path

# Define paths to your datasets
dataset_paths = [
    './Fruit-262/',
    './fruits/',
    './fruits_dataset/',
    './fruits-360/'
]

# Define the combined dataset path
combined_dataset_path = './combined_fruits_dataset/'

# Define subdirectories for train, test, and val
subdirs = ['train', 'test', 'val']

# Create the combined dataset directory structure
for subdir in subdirs:
    os.makedirs(os.path.join(combined_dataset_path, subdir), exist_ok=True)

# Function to copy images from source to destination
def copy_images(src_dir, dest_dir):
    for class_dir in os.listdir(src_dir):
        class_path = os.path.join(src_dir, class_dir)
        if os.path.isdir(class_path):
            dest_class_path = os.path.join(dest_dir, class_dir)
            os.makedirs(dest_class_path, exist_ok=True)
            for img_file in os.listdir(class_path):
                src_img_path = os.path.join(class_path, img_file)
                dest_img_path = os.path.join(dest_class_path, img_file)
                shutil.copy(src_img_path, dest_img_path)

# Copy images from each dataset to the combined dataset
for dataset_path in dataset_paths:
    for subdir in subdirs:
        src_subdir_path = os.path.join(dataset_path, subdir)
        if os.path.exists(src_subdir_path):
            dest_subdir_path = os.path.join(combined_dataset_path, subdir)
            copy_images(src_subdir_path, dest_subdir_path)

print("Datasets combined successfully!")

Datasets combined successfully!
